In [117]:
import pandas as pd
import numpy as np
import random

In [2]:
def clean_census_frame(file_path):
    '''
    input) file_path
        >> path to csv
    '''
    # load data
    df = pd.read_csv( file_path , low_memory=False )

    # and copy
    _df = df.copy()

    # reset column names to current 0th row values
    _df.columns = _df.iloc[0]
    # new 2000 dataframe without row where values are from
    _df = _df[1:]
    # reset index
    clean_df = _df.reset_index()
    
    return clean_df

In [15]:
# 2011
y2k11 = clean_census_frame('../data/acs/aff_download/ACS_11_5YR_DP05_with_ann.csv')
# 2012
y2k12 = clean_census_frame('../data/acs/aff_download/ACS_12_5YR_DP05_with_ann.csv')
#2013
y2k13 = clean_census_frame('../data/acs/aff_download/ACS_13_5YR_DP05_with_ann.csv')
# 2014
y2k14 = clean_census_frame('../data/acs/aff_download/ACS_14_5YR_DP05_with_ann.csv')
# 2015
y2k15 = clean_census_frame('../data/acs/aff_download/ACS_15_5YR_DP05_with_ann.csv')
# 2016
y2k16 = clean_census_frame('../data/acs/aff_download/ACS_16_5YR_DP05_with_ann.csv')
# 2017
y2k17 = clean_census_frame('../data/acs/aff_download/ACS_17_5YR_DP05_with_ann.csv')

In [100]:
# gather column names
column_names = [_ for _ in y2k11[:0]]
column_names[3]

'Geography'

In [19]:
for _ in [y2k11,y2k12,y2k13,y2k14,y2k15,y2k16,y2k17]:
    print(_.info(),'\n\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33120 entries, 0 to 33119
Columns: 328 entries, index to Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units
dtypes: int64(1), object(327)
memory usage: 82.9+ MB
None 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33120 entries, 0 to 33119
Columns: 328 entries, index to Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units
dtypes: int64(1), object(327)
memory usage: 82.9+ MB
None 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33120 entries, 0 to 33119
Columns: 328 entries, index to Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units
dtypes: int64(1), object(327)
memory usage: 82.9+ MB
None 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33120 entries, 0 to 33119
Columns: 328 entries, index to Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units
dtypes: int64(1), object(327)
memory usage: 82.9+ MB
None 


<class 'pandas.core.frame.Da

In [142]:
# make dataframe for each zip code, should contain row from each year
def frame_per_zip(dataframes):
    '''
    take dataframe for each year set is available
    find common zipcode for each year
    make dataframe for that zipcode
        containing each year's measurements
    note:
        dataframes must be same length
    '''
    # check length of dataframes is same
    for _ in range(len(dataframes)):
        # done to ensure each zip code has same represnetation 
        if len(dataframes[_-1]) != len(dataframes[_]):
            # stop if we have different lengths
            raise Exception(f'len(dataframe[{-_}]) != len(dataframe[{_}])')
        # also check that dataframes have same Id
        random_samples = [random.randint(1,int(len(dataframes[-_])/2)),
                          random.randint(1,int(len(dataframes[-_]))),
                          random.randint(int(len(dataframes[-_])/2),len(dataframes[-_]))]
        for sample in random_samples:
            # pull Id from df to compare
            if dataframes[_]['Id'][sample] != dataframes[_-2]['Id'][sample]:
                # stop if they don't match
                raise Exception(f"NON MATCHING Id\n{dataframes[_].Id[sample]}\nERROR\n{dataframes[_-2].Id[sample]}\n")
    
    # now we can get to work
    mini_dfs = []
    for i in range(len(dataframes[0])):
        mini_df = pd.DataFrame(index=['2011','2012','2013','2014','2015','2016','2017'], columns=dataframes[0].columns)
        for _ in range(len(dataframes)):
            mini_df.iloc[_] = dataframes[_][i]
        mini_dfs.append(mini_df)
        
    

In [141]:
frame_per_zip([y2k11,y2k12,y2k13,y2k14,y2k15,y2k16,y2k17])
# mini_df = pd.DataFrame(index=['2011','2012','2013','2014','2015','2016','2017'], columns=['a','b','c','d','e'])
# mini_df.iloc[1]

a    NaN
b    NaN
c    NaN
d    NaN
e    NaN
Name: 2012, dtype: object

In [137]:
# dataframes = [y2k11,y2k12,y2k13,y2k14,y2k15,y2k16,y2k17]
# mini_df = pd.DataFrame(index=['2011','2012','2013','2014','2015','2016','2017'], columns=dataframes[0].columns)
# mini_df.iloc[0] = 'lol'
# mini_df.iloc[0]
# mini_df

In [5]:
'''# load 2000 data
y2k = pd.read_csv( a , low_memory=False )
# load 2010 data
y2k10 = pd.read_csv( b , low_memory=False )

# 2000 Census
b = y2k.copy()
# 2010 Census
o = y2k10.copy()

# reset 2000 columns to current 0th row values
b.columns = b.iloc[0]
# new 2000 dataframe without row where values are from
b = b[1:]
# reset index
b = b.reset_index()

# reset 2010 columns to current 0th row values
o.columns = o.iloc[0]
# new 2010 dataframe without row where values are from
o = o[1:]
# reset index
o = o.reset_index()

# identify zip codes from 2000 .Geography (last 5 chars of string)
zip_2000_codes = [q[-5:] for q in b.Geography]  # ValueError: invalid literal for int() with base 10: '006HH'
# identify zip codes from 2010 .Geography (last 5 chars of string)
zip_2010_codes = [q[-5:] for q in o.Geography]

# from 2000.Geography , instance is not seen in 2010.Geography  -- sample: zip_code = (2, 'c')
in_2000_but_not_2010_from_2000 = [zip_code for zip_code in enumerate(zip_2000_codes) if zip_code[1] not in zip_2010_codes]
# from 2010.Geography , instance is not seen in 2000.Geography  -- sample: zip_code[1] = 'c'
in_2010_but_not_2000_from_2010 = [zip_code for zip_code in enumerate(zip_2010_codes) if zip_code[1] not in zip_2000_codes]

# from 2000.Geography , instance is seen in 2010.Geography
in_2000_and_2010_from_2000 = [zip_code for zip_code in enumerate(zip_2000_codes) if zip_code[1] in zip_2010_codes]
# from 2010.Geography , instance is seen in 2000.Geography
in_2010_and_2000_from_2010 = [zip_code for zip_code in enumerate(zip_2010_codes) if zip_code[1] in zip_2000_codes]

# index of objects coexisting in 2000 and 2010
of_2000_indexes = [i for i,j in in_2000_and_2010_from_2000]
# index of objects coexisting in 2010 and 2000 
of_2010_indexes = [i for i,j in in_2010_and_2000_from_2010]
# ^note: these are different lists, if took j instead of i, then would be same list
if [j for i,j in in_2000_and_2010_from_2000] != [j for i,j in in_2010_and_2000_from_2010]:
    # like is seen here, j for j == True
    raise Exception(f'FLAWED ASSUMPTION , [j for i,j in 2000] != [j for i,j in 2010]\n'
                    f'len {len(in_2000_and_2010_from_2000)} {len(in_2010_and_2000_from_2010)}')
# however i for i == False
if of_2000_indexes == of_2010_indexes:
    # cheers
    raise Exception('FLAWED ASSUMPTION , of_2000_indexes != of_2010_indexes\n'
                    f'len y2k {len(of_2000_indexes)} 2k10 {len(of_2010_indexes)}')  

# thin 2000 to shared geo
b = b.iloc[of_2000_indexes]
# thin 2010 to shared geo
o = o.iloc[of_2010_indexes]'''
pass